In [2]:
from dataclasses import dataclass
import numpy as np
import pandas as pd
import numpy_financial as npf

@dataclass
class ModelInputs:
    price_machine: float = 1000000
    loan_life: int = 5
    initial_default: float = 0.3
    default_decay: float = 0.9
    final_default: float = 0.4
    recovery_rate: float = 0.4
    interest: float = 0.2
    num_iterations: int = 1000
    default_std: float = 0.05  # Standard deviation for default probability
    recovery_std: float = 0.02  # Standard deviation for recovery rate
        
model_data = ModelInputs()

# Now use these new attributes in your simulation
def simulate_cash_flows_with_randomness(model_inputs):
    irrs = []  # List to store IRRs from each iteration
    
    for _ in range(model_inputs.num_iterations):
        # Simulate random default probabilities and recovery rates
        default_probs = np.random.normal(
            model_inputs.initial_default, 
            model_inputs.default_std, 
            model_inputs.loan_life
        )
        # Ensure the probabilities are within the specified range
        default_probs = np.clip(default_probs, a_min=model_inputs.initial_default, a_max=model_inputs.final_default)
        
        recovery_rates = np.random.normal(
            model_inputs.recovery_rate, 
            model_inputs.recovery_std, 
            model_inputs.loan_life
        )
        # Ensure the recovery rates are within the range [0, 1]
        recovery_rates = np.clip(recovery_rates, a_min=0, a_max=1)
        
        cash_flows = [-model_inputs.price_machine]
        for year in range(model_inputs.loan_life):
            expected_cash_flow = model_inputs.price_machine * model_inputs.interest * (1 - default_probs[year]) + \
                                 model_inputs.price_machine * default_probs[year] * recovery_rates[year]
            cash_flows.append(expected_cash_flow)
        
        irr = npf.irr(cash_flows)  # Calculate the IRR
        irrs.append(irr)
    
    return np.mean(irrs)  # Return the average IRR over all iterations

# Running the simulation with randomness
interest_rates = [0.05, 0.1, 0.15, 0.2]  # Example interest rates
results = []

for rate in interest_rates:
    # Set the interest rate in the model data
    model_data.interest = rate
    # Get the average IRR with randomness introduced
    average_irr = simulate_cash_flows_with_randomness(model_data)
    results.append([rate, model_data.loan_life, model_data.initial_default, average_irr])

irr_df = pd.DataFrame(results, columns=['Interest Rate', 'Loan Life', 'Initial Default Probability', 'Average IRR'])
irr_df


ModuleNotFoundError: No module named 'dataclasses'